Select 264 random clips in total (considering 33 classes) or 8 random clips per class (decide how many classes to account for, for example are we interested in improve and validate clips for rooster? or for noise? 

Global Random Sampling:

Perform random sampling across all remaining predictions without class constraints.
Adjust sampling to exclude clips belonging to any unwanted classes (e.g., Gallus_gallus), if needed in the future.

### Draft for Random Sampling Logic

In [ ]:
import pandas as pd
from pathlib import Path
import random

# Paths
big_df_path = "/path/to/big_df.csv"
tracker_path = "/path/to/training_set_tracker.csv"
output_dir = "/path/to/random_sampled_clips/"
iteration_number = 1  # Current iteration number
random_clip_count = 200  # Number of random clips to sample

# Load data
big_df = pd.read_csv(big_df_path)
training_set_tracker = pd.read_csv(tracker_path)

# Filter out unwanted classes
unwanted_classes = ["Environmental", "Noise"]
filtered_df = big_df[~big_df["class"].isin(unwanted_classes)]

# Exclude already used clips
used_clips = training_set_tracker["filename"].tolist()
filtered_df = filtered_df[~filtered_df["filename"].isin(used_clips)]

# Randomly sample clips
random_samples = filtered_df.sample(min(random_clip_count, len(filtered_df)), random_state=42)

# Save selected random clips
for _, row in random_samples.iterrows():
    class_name = row["class"]
    class_dir = Path(output_dir) / class_name
    class_dir.mkdir(parents=True, exist_ok=True)
    
    filename = row["filename"]
    offset = int(row["offset"])
    score = row["score"]
    output_filename = f"{score:.2f}_{offset}_{filename}.wav"

    # Assuming clips are extracted similarly as before
    audio_path = Path("/path/to/audio/files/") / f"{filename}.wav"
    try:
        audio = Audio.from_file(audio_path, offset=offset, duration=3)
        audio.save(class_dir / output_filename)
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

print("Random sampling complete. Clips saved.")
